<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>

<html><font size=4><B>1. Κατασκευή γράφου και προεργασία για την πρόβλεψη συνδέσμων</B></font>
    <li><B>1(i).</B> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/dbpedia-similar/"> Similarities (DBpedia)</a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$ <a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]</a>,<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]</a>. Οι κόμβοι του γράφου αναπαριστούν σελίδες της Wikipedia και οι ακμές σχέση ομοιότητας μεταξύ τους. Στην σελίδα <a href="http://konect.cc/networks/dbpedia-similar/"> KONECT (DBpedia)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a><BR>
        <B><U>Υπόδειξη:</U></B> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί, στην ερώτηση 2.<BR></html>

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import networkx as nx

p = Path("./data/out.dbpedia-similar")
a_file = open(p)

next(a_file)
next(a_file)

edgelist = []
for line in a_file:
    line = tuple(map(int,line.strip().split()))
    edgelist.append(line)

G = nx.Graph()
G.add_edges_from(edgelist)
G.remove_edges_from(nx.selfloop_edges(G))

In [2]:
# to_numpy_array requires nodelist to work properly
# nx.to_numpy_array(G, nodelist=sorted(G.nodes()))

In [3]:
def find_degree(G):
    return [G.degree(n) for n in G.nodes()]

 <li><B>1(ii).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li><BR>

In [4]:
def describe_graph(H):
    initial_node_count= len(H.nodes())
    edge_count = H.number_of_edges()
    initial_connected_components= list(nx.connected_components(H))
    print("Initial node count is {}".format(initial_node_count))
    print("Edges in G are {}".format(edge_count))
    print("Mean node degree of G is {:.3f}".format(np.mean(find_degree(H))))
    print('Intial connected components are {}'.format(len((initial_connected_components))))
    return initial_node_count, edge_count, initial_connected_components


In [5]:
initial_node_count,edge_count, initial_connected_components =  describe_graph(G)

Initial node count is 430
Edges in G are 564
Mean node degree of G is 2.623
Intial connected components are 95


<li><B>1(iii). </B>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li><BR>

In [6]:
def find_unconnected_pairs(G):
    
    #remove self loops
    G.remove_edges_from(nx.selfloop_edges(G))
    # transform to adjacency matrix representation
    adj_mat = nx.to_numpy_array(G, nodelist=sorted(G.nodes()))
    unconnected_pairs = []
    for row_idx, row in enumerate(adj_mat):
        for col_idx, column in enumerate(row):
            # stop if column==row (lower-triangular)
            if col_idx==row_idx:
                break
            if column==0:
                # add to unconnected pairs (indexes + 1 (compensate for 0-index))
                unconnected_pairs.append((row_idx+1,col_idx+1))
    return unconnected_pairs


In [7]:
unconnected_pairs = find_unconnected_pairs(G)

In [8]:
print(f"There are {len(unconnected_pairs)} unconnected pairs in G.")

There are 91671 unconnected pairs in G.


<li><B>1(iv).</B> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li><BR>

In [9]:
def find_removable_edges(G):
    # remove self loops
    G.remove_edges_from(list(nx.selfloop_edges(G)))
    removable_edges = []
    # Copy the graph
    D = G.copy()
    # For each edge of the original graph
    for edge in G.edges():
        # Make a copy of D
        H = D.copy()
        # Remove in place the edge
        H.remove_edge(*edge)
        # Check if the conditions hold
        if nx.number_of_nodes(H)==initial_node_count and (nx.number_connected_components(H)==len(initial_connected_components)):
            # Remove edge from G's copy 
            D.remove_edge(*edge)
            removable_edges.append(edge)
    return removable_edges

In [10]:
removable_edges = find_removable_edges(G)

In [11]:
print(f"There are {len(removable_edges)} removable edges in G.")

There are 229 removable edges in G.


<li><B>1(vα).</B> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 1(iii) και 1(iv) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable). <BR>
    <li><B>1(vβ).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 1(iv). <BR>
    <li><B>1(vγ).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li><BR>

In [12]:
df_pos = pd.DataFrame(removable_edges)
#dataframe of removable edges (they will function as positive samples)
df_pos['link'] = 1  #uncomment and complete

df_neg = pd.DataFrame(unconnected_pairs)
#dataframe of unconnected node pairs (they will function as negative samples)
df_neg['link'] = 0 # uncomment and complete

#merge in one dataframe
df = pd.concat([df_pos,df_neg])
df.columns = ["u","v","link"]

# df to dictionary with composite key (u,v)
edges_link_dict = {(tup.u,tup.v): tup.link for _,tup in df.iterrows()}

#build graph G'=G_reduced
G_prime = G.copy()
G_prime.remove_edges_from(removable_edges)
_,g_prime_edge_count,_= describe_graph(G_prime)

Initial node count is 430
Edges in G are 335
Mean node degree of G is 1.558
Intial connected components are 95


<html><font size=4><B>2. Εισαγωγή σε similarity-based μετρικές για Link prediction</B></font><BR><BR>
     <li> Η μετρική <B>Jaccard Coefficient (JC)</B> των κόμβων ${u}$ και ${v}$ ορίζεται ως $\frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u) \cup \Gamma(v)|}$, όπου ${\Gamma(u)}$ είναι το σύνολο των γειτονικών κόμβων του κόμβου ${u}$.  <B></B>
        <li> Η μετρική <B>Preferential Αttachment (PA)</B> των κόμβων $u$ και $v$ ορίζεται ως $|\Gamma(u)| |\Gamma(v)|$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.</li>
        <li> Η μετρική <B>Resource Allocation (RA)</B> των κόμβων $u$ και $v$ ορίζεται ως $\sum_{w \in \Gamma(u) \cap \Gamma(v)} \frac{1}{|\Gamma(w)|}$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.</li><BR><BR>
    <li><B>2(i).</B> Για τον γράφο $G'$, που κατασκευάσατε στο ερώτημα 1(vβ), υπολογίστε την μετρική <B>Jaccard Coefficient (JC)</B>.  <u><a href="https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html?highlight=link%20prediction">Χρησιμοποιήστε την υλοποιημένη συνάρτηση του λογισμικού NetworkX.<a></u> <BR><BR>
    <li><B>2(ii). </B>Δώστε τις $k$ ακμές με την υψηλότερη τιμή μετρικής $JC$, όπου $k$ ο $Α.Μ.$ σας $mod$ $10$. Για $Α.Μ.$ λήγοντα σε $0$ δώστε τις $10$ ακμές με την υψηλότερη τιμή μετρικής.
        (Για την σύγκριση μεταξύ διαφορετικών μετρικών προτιμάται η εύρεση των $k$ επικρατέστερων συνδέσμων, αφού οι μετρικές επιστρέφουν διαφορετικό εύρος τιμών.)</li><BR>
    <li><B>2(iii). </B>Προσδιορίστε για την μέτρηση της απόδοσης της μετρικής $JC$ τα μεγέθη $Precision$, $Recall$ και $Accuracy$ και αναπαραστήστε τα γραφικά για τιμές κατωφλίου: $0.1$, $0.15$, $0.25$, $0.5$, $1.0$.  <BR><BR>
    $Precision=\frac{TP}{TP+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα <BR>
    $Recall=\frac{TP}{TP+FN}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα</li> 
    $Accuracy=\frac{TP+TN}{TP+FN+TN+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $TN$: αληθώς αρνητικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα</li><BR><BR>
        <B><U>Υπόδειξη:</U></B>
        Θετικό θεωρήστε ένα αποτέλεσμα, αν η μετρική $JC$ υπερβαίνει μια τιμή κατωφλίου (>=). Αρνητικό θεωρήστε το στην αντίθετη περίπτωση.
        <li>Αληθώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά (η τιμή της $JC$ υπερβαίνει την τιμή κατωφλίου) και πράγματι θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iv)).</li>
        <li>Αληθώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά και πράγματι δεν θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iii)).</li>
        <li>Ψευδώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά, ενώ δεν θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Ψευδώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά, ενώ θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Το άθροισμα $TN+FP$ πρέπει κάθε φορά να σας δίνει όλες τις ασύνδετες ακμές του γράφου $G$.</li>   
<li>Από το ερώτημα 1(iii) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες δεν θα υπάρξουν συνδέσεις.</li> <li>Από το ερώτημα 1(iv) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες θα υπάρξουν συνδέσεις.</li><BR></html>

In [13]:
useful_edges = removable_edges+unconnected_pairs

In [14]:
from sklearn.metrics import recall_score, precision_score, accuracy_score
MAX_EDGES = len(useful_edges)

def top_k_jc(G,k,X):
    jaccard_coeffs = []
    # find all jaccard coeffs for given graph and edges
    for i,(u,v,jc) in enumerate(nx.link_prediction.jaccard_coefficient(G,X)):
        jaccard_coeffs.append((u,v,jc))
    # Sort based on jaccard coeff
    jaccard_coeffs = sorted(jaccard_coeffs, key=lambda x: x[2], reverse=True)[:k]
    return jaccard_coeffs

def JCfunction(threshold_jc,G,links_dict,X):

    preds = np.empty(MAX_EDGES)
    truth = np.empty(MAX_EDGES)
    
    jaccard_coeffs = []
    # find all jaccard coeffs for given graph and edges
    for i,(u,v,jc) in enumerate(nx.link_prediction.jaccard_coefficient(G,X)):
        
        truth[i] = links_dict[(u,v)]
        preds[i] = 1 if jc>=threshold_jc else 0
    
    recall_jc = recall_score(truth,preds)
    precision_jc = precision_score(truth,preds)
    accuracy_jc = accuracy_score(truth,preds)

    return recall_jc,precision_jc, accuracy_jc;


In [15]:
k = 3400098 % 10
top_jcs = top_k_jc(G_prime,k,useful_edges)
top_jcs

[(161, 162, 1.0),
 (105, 106, 1.0),
 (105, 109, 1.0),
 (119, 121, 1.0),
 (141, 142, 1.0),
 (186, 187, 1.0),
 (332, 333, 1.0),
 (332, 334, 1.0)]

# Τοπ-k Jaccard Coefficients

Όπως βλέπουμε το ποίες είναι οι ακμές με το μεγαλύτερο JC δεν έχει τόση σημασία
αλλά παρατηρούμε 

ότι όλοι οι κορυφαίοι κ έχουν jaccard=1 που σημαίνει οτι ο σύνολο
της ένωσης των γειτόνων των κομβων ταυτίζεται με το σύνολο της τομής

In [16]:
jc_performance = np.empty(shape=(5,4))
thresholds = [0.1 , 0.15, 0.25, 0.5, 1.0]

for i,thold in enumerate(thresholds):
    rc,prec,acc = JCfunction(thold,G_prime,edges_link_dict, useful_edges )
    jc_performance[i] = thold,rc,prec,acc

# Threshold 

Για τα διάφορα κατώφλια βλέπουμε παρακάτω την απόδοση του απλού αυτόυ link-"prediction" algorithm.

Στην πρώτη στήλη έχουμε το κατώφλι ενω στις επόμενες τα recall, precision, accuracy.


Όπως έιναι αναμενόμενο παρατηρούνται τα εξής με την άνοδο του κατωφλίου:

Μείωνεται το recall $\frac{\text{TP}}{\text{TP+FN}}$ καθώς, όντας πιο "αυστηροί" με τα θετικά μειώνουμε τα TP και αυξάνουμε τα FN

(εμπειρικά από όλα τα θετικά δείγματα "ενθυμούμαστε" λίγοτερα).

Αντιθέτως έχουμε αύξηση στο precision καθώς είμαστε πιο σίγουροι (confident) στα θετικά μας δείγματα.

Τέλος παρατηρούμε ότι στο accuracy έχουμε μια σχετικά σταθερή αλλα ελαφρώς ανοδική πορεία.

Αυτό επιβεβαιώνει
το γεγονός ότι το imbalance στο dataset οδήγει τον "predictor" μας να πρόβλεπει σχεδόν συνεχώς 0 επιτυγχάνοντας πολύ υψηλά ποσοστά ακρίβειας.

In [17]:
jc_performance

array([[0.1       , 0.70742358, 0.32079208, 0.99553863],
       [0.15      , 0.68995633, 0.32443532, 0.99564744],
       [0.25      , 0.6768559 , 0.34598214, 0.99600653],
       [0.5       , 0.60262009, 0.41691843, 0.99690968],
       [1.        , 0.55021834, 0.51012146, 0.99756257]])

<html><font size=4><B>3. Πρόβλεψη συνδέσμων βάσει similarity-based μετρικών</B></font>    
        <li><B>3(i).</B> Προτείνετε μια τουλάχιστον μέθοδο, για να εξερευνήσουμε και να εκφράσουμε μέσα από τα πακέτα NumPy ή SciPy ή Pandas την συσχέτιση (correlation) μεταξύ των τιμών των μετρικών <B>Jaccard Coefficient</B>, <B>Preferential Attachment</B>, <B>Resource Allocation</B> για τον μειωμένο γράφο $G'$. Σημ.: Δεν χρειάζεται κάποια υλοποίηση, μόνο αναφορά της μεθόδου συσχέτισης και επεξήγηση της επιλογής σας.
     <li><B>3(ii).</B> Εργαστείτε πάνω σε <B><U>αντίγραφο</U></B> του dataframe που κατασκευάσατε στο ερώτημα 1(vα) (unconnected+removables). Δημιουργήστε στήλη (column) με τις ακμές του γράφου και θέστε την ως index του dataframe. Στο dataframe θα προσθέσετε ως στήλες τις τιμές των μετρικών (<B>Jaccard Coefficient</B>, <B>Preferential Attachment</B>, <B>Resource Allocation</B>) για τον μειωμένο γράφο $G'$. Οι τιμές των μετρικών μπορούν να υπολογιστούν με τις συναρτήσεις του πακέτου NetworkX.</li>   </font></html>
    
    |       | link  |Jacc.Coeff.|Pr.Attachment| Resource Allocation|
    |-------|-------|-----------|-------------|--------------------|
    |(1, 6) |  0    |  0.333333 |     32      |    0.700000        |
    |(1, 46)|  0    |  0.200000 |      8      |    0.166667        |

# Συσχέτιση Μετρικών

Για το εν λόγω ερώτημα μια μονοσήμαντη απάντηση είναι δύσκολη. Προσωπικά θεωρώ πως θα έπρεπε να γίνει η πειραματική χρήση

πολλαπλών μέτρων συσχέτισης όπως Pearson, Kendall, Spearman και να εξεταστεί ποιά απο τίς παραπάνω αποδίδει καλύτερα.

Φυσικά κάθε μέτρικη συσχέτισης έχει κάποιες προϋποθέσεις πχ. η Pearson υποθέτει ότι και οι δύο μεταβλητές ακολουθούν κανονική κατανομή, έχουν γραμμική σχέση μεταξύ τους
καθώς και ομοσκεδαστικότητα.

Αυτές οι υποθέσεις αν και αυστηρές πρέπει να ελεγχθούν προκειμένου να διαλέξουμε την Pearson για την εκφραση της συσχετισης μιας και αν παραβιάζονται μπορεί να οδηγήσουν
σε επισφαλή αποτελέσματα.


In [18]:
#replicate of the dataframe "unconnected+removables"---dataframe will be used again in (4)

sim_metrics_dict = {}


jc_pred = nx.link_prediction.jaccard_coefficient(G_prime, removable_edges)
pa_pred = nx.link_prediction.preferential_attachment(G_prime,removable_edges)
ra_pred = nx.link_prediction.resource_allocation_index(G_prime,removable_edges)

# iterate over JC,Pr.Attachment, Resource Allocation generators (for existing links)
for (u,v,jc),(_,_,pa),(_,_,ra) in zip(jc_pred,pa_pred,ra_pred):
    sim_metrics_dict[(u,v)] = (1,jc,pa,ra)
    

jc_pred = nx.link_prediction.jaccard_coefficient(G_prime, unconnected_pairs)
pa_pred = nx.link_prediction.preferential_attachment(G_prime,unconnected_pairs)
ra_pred = nx.link_prediction.resource_allocation_index(G_prime,unconnected_pairs)

# iterate over JC,Pr.Attachment, Resource Allocation generators (for non-existing links)
for (u,v,jc),(_,_,pa),(_,_,ra) in zip(jc_pred,pa_pred,ra_pred):
    sim_metrics_dict[(u,v)] = (0,jc,pa,ra)

    
df_expanded = pd.DataFrame.from_dict(sim_metrics_dict, orient="index")
df_expanded.columns = ["link","Jacc.Coeff.","Pr.Attachment","ResourceAllocation"]

In [19]:
df_expanded
#df_expanded.loc[[(1,2)]]

,link,Jacc.Coeff.,Pr.Attachment,ResourceAllocation
"(1, 2)",1,0.0,1,0.000000
"(1, 3)",1,0.0,2,0.000000
"(1, 4)",1,0.0,1,0.000000
"(3, 4)",1,0.5,2,0.142857
"(3, 5)",1,0.0,4,0.000000
...,...,...,...,...
"(430, 424)",0,0.0,1,0.000000
"(430, 425)",0,0.0,1,0.000000
"(430, 426)",0,0.0,1,0.000000
"(430, 427)",0,0.0,1,0.000000


In [20]:
#dataframe for similarity-based measurement values
#Jaccard Coefficient
jacc = df_expanded['Jacc.Coeff.']
#Preferential Attachment
pr_at = df_expanded['Pr.Attachment']
#Resource Allocation
res_al = df_expanded["ResourceAllocation"]

<li><B>3(iiiα).</B> Χρησιμοποιήστε τον Random Forest Classifier και το αντίγραφο dataframe που κατασκευάσατε στο ερώτημα 3(ii), για να προβλέψετε συνδέσεις. Συμπληρώστε στον κώδικα: τα features ($X$), την target variable ($y$) και τα ορίσματα της συνάρτησης $fit$ για το training του μοντέλου.</li>
<li><B>3(iiiβ).</B> Δώστε τις ($A.M$ $mod$ $10$) πρώτες προβλέψεις και την ακρίβεια ($accuracy$) του μοντέλου. </li>
<li><B>3(iiiγ).</B>Υπολογίστε την πιθανότητα να συνδεθούν οι κόμβοι ($1$,$47$) βάσει του μοντέλου σας. </li>  

In [21]:
#uncomment whole cell
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
 
X = df_expanded[["Jacc.Coeff.","Pr.Attachment","ResourceAllocation"]]  #features, uncomment and complete
y = df_expanded['link'] #target, uncomment and complete

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0) #uncomment and complete

In [22]:
#uncomment whole cell
classifier = RandomForestClassifier()   

from sklearn.preprocessing import MinMaxScaler 
mm_scaler = MinMaxScaler() 
X_train_scaled = mm_scaler.fit_transform(X_train)
X_test_scaled = mm_scaler.transform(X_test)

parameters = {'n_estimators' : [10,50,100], 'max_depth' : [5,10,15]} 
model = GridSearchCV(classifier, param_grid = parameters) 

In [23]:
model.fit(X_train_scaled,y_train )#train the model, uncomment and complete

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'n_estimators': [10, 50, 100]})

In [24]:
predictions = model.predict(X_test_scaled)       #uncomment
predictions[:8] #get (A.M. last digit) first predictions, uncomment and complete

array([0, 0, 0, 0, 0, 0, 0, 0])

Το σκορ του μοντέλου φαίνεται παρακάτω 

In [25]:
model.score(X_test_scaled,y_test)

0.9988248095756257

In [26]:
df_expanded.loc[[(47,1)]]

,link,Jacc.Coeff.,Pr.Attachment,ResourceAllocation
"(47, 1)",0,0.0,1,0.0


Προκειμένου να προβλέψουμε την τιμή για την ακμή (47,1) πρέπει πρώτα να περάσουμε τις 
αντίστοιχες στήλες του df απο τον scaler του μοντέλου

και ύστερα να χρησιμοποιήσουμε
την predict_proba (για να πάρουμε πιθανότητες για κάθε μια απο τις κλάσεις).

Όπως φαίνεται παρακάτω, το μοντέλο είναι παρα πολύ σίγουρο (99.98%) ότι δεν υπάρχει ακμή μεταξύ των κόμβων 47 και 1

In [27]:
predict_proba = model.predict_proba(mm_scaler.transform(df_expanded.loc[[(47,1)],['Jacc.Coeff.','Pr.Attachment','ResourceAllocation']]))

In [28]:
predict_proba

array([[9.99803616e-01, 1.96384164e-04]])

<html><font size=4><B>4. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </B></font><BR><BR>
    <li><B>4(i).</B> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <B>Node2vec</B> <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <BR>Για την επιλογή των παραμέτρων: 
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

In [29]:
#install node2vec with conda / pip
from node2vec import Node2Vec   #uncomment
import multiprocessing          #uncomment
import gensim
# #generate walks
node2vec_walker = Node2Vec(G_prime,dimensions=128 ,p=1 ,q=1,num_walks=10 ,walk_length=80 ,workers=multiprocessing.cpu_count()) #uncomment and complete

# train node2vec model
n2v_model = gensim.models.Word2Vec(node2vec_walker.walks, workers=node2vec_walker.workers, sg=1)

Computing transition probabilities: 100%|██████████| 430/430 [00:00<00:00, 39938.68it/s]


In [30]:
df_expanded2 =df_expanded.copy()
df_expanded2.reset_index(inplace=True)
df_expanded2[["source","target"]] = df_expanded2['index'].apply(pd.Series)

In [31]:
df_expanded2

,index,link,Jacc.Coeff.,Pr.Attachment,ResourceAllocation,source,target
0,"(1, 2)",1,0.0,1,0.000000,1,2
1,"(1, 3)",1,0.0,2,0.000000,1,3
2,"(1, 4)",1,0.0,1,0.000000,1,4
3,"(3, 4)",1,0.5,2,0.142857,3,4
4,"(3, 5)",1,0.0,4,0.000000,3,5
...,...,...,...,...,...,...,...
91895,"(430, 424)",0,0.0,1,0.000000,430,424
91896,"(430, 425)",0,0.0,1,0.000000,430,425
91897,"(430, 426)",0,0.0,1,0.000000,430,426
91898,"(430, 427)",0,0.0,1,0.000000,430,427


<html><font>
    <li><B>4(ii).</B> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><BR>
    <li><B>4(iii).</B> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><BR></font></html>

In [32]:
#apply trained Node2Vec model on each node pair of the dataframe "unconnected+removables"
#uncomment:
x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(df_expanded2['source'], df_expanded2['target'])]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, df_expanded2['link'], random_state = 0)  #uncomment and complete

In [34]:
#uncomment whole cell
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(max_depth=5, random_state=0) 
classifier.fit(X_train,y_train)   #uncomment and complete

RandomForestClassifier(max_depth=5, random_state=0)

In [35]:
accuracy_score(classifier.predict(X_test),y_test)

0.9986071817192601

In [36]:
import sklearn 
sklearn.metrics.confusion_matrix(y_test, classifier.predict(X_test))

array([[22918,     0],
       [   32,    25]])

<html><font size=4> <B><U>Ερώτηση bonus (+10/100)</U><B>: </font>Για τους binary operators για learning των edge features με τον αλγόριθμο Node2Vec που αναφέρονται στο <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a> ((α) Average, (β) Hadamard, (γ) Weighted-L1 και (δ) Weighted-L2) τροποποιήστε τoν κώδικά σας και σημειώστε τις παρατηρήσεις σας ως προς την απόδοση του αλγορίθμου.</font></html>

<html><font size=4><B><U>Βιβλιογραφία:</U></B><BR></font>
<a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]	Jérôme Kunegis. KONECT – The Koblenz Network Collection. In Proc. Int. Conf. on World Wide Web Companion, pages 1343–1350, 2013.</a><BR>
<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]	Sören Auer, Christian Bizer, Georgi Kobilarov, Jens Lehmann, Richard Cyganiak, and Zachary Ives. DBpedia: A nucleus for a web of open data. In Proc. Int. Semant. Web Conf., pages 722–735, 2008.<BR>
<a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3] A. Grover, J. Leskovec, Node2vec: Scalable Feature Learning for Networks. ACM KDD, 2016.</a><BR>
 <a href="https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf">[4] T.Mikolov, I.Sutskever, K.Chen, G.S.Corrado and J.Dean, Distributed representations of words and phrases and their compositionality. In NIPS, 2013.</a><BR></font></html>

# Average Embedder
Όπως φαίνεται παρακάτω τα αποτελέσματα δεν επηρεάζονται καθως στην αρχική περίπτωση 
χρησιμοποιούσαμε το άθροισμα των vectors και τωρα χρησιμοποιουμε τον μέσο των vectors.

Αυτό σε συνδυασμό με την χρήση του random forest classifier οδηγεί σε όμοια (scaled) κρίτηρια επιλογής του αλγορίθμου
με αποτέλεσμα το confusion matrix να μένει το ίδιο.


In [37]:
# Average embeddings
from node2vec.edges import AverageEmbedder

avg_edges_embs = AverageEmbedder(keyed_vectors=n2v_model.wv)

x = [avg_edges_embs[(str(i),str(j))] for i,j in zip(df_expanded2['source'], df_expanded2['target'])]
X_train, X_test, y_train, y_test = train_test_split(x, df_expanded2['link'], random_state = 0)  #uncomment and complete

classifier = RandomForestClassifier(max_depth=5, random_state=0) 
classifier.fit(X_train,y_train)   #uncomment and complete

sklearn.metrics.confusion_matrix(y_test, classifier.predict(X_test))

array([[22918,     0],
       [   32,    25]])

# Hadamard Embedder

Από τα παρακάτω αποτελέσματα παρατηρούμε μια θετική αλλαγή καθώς έχουμε περισσότερα αληθώς αρνητικά(ύπαρξη ακμής) (που είναι και τα πιο σημαντικά λόγω του class imbalance).

Από αυτό αλλά και από το paper που παρατέθηκε βλέπουμε ότι η χρήση του hadamard είναι οδηγεί σε πιο robust κριτήρια απόφασης.

In [38]:
# Hadamard embeddings
from node2vec.edges import HadamardEmbedder

had_edges_embs = HadamardEmbedder(keyed_vectors=n2v_model.wv)

x = [had_edges_embs[(str(i),str(j))] for i,j in zip(df_expanded2['source'], df_expanded2['target'])]
X_train, X_test, y_train, y_test = train_test_split(x, df_expanded2['link'], random_state = 0)  #uncomment and complete

classifier = RandomForestClassifier(max_depth=5, random_state=0) 
classifier.fit(X_train,y_train)   #uncomment and complete

sklearn.metrics.confusion_matrix(y_test, classifier.predict(X_test))

array([[22918,     0],
       [   28,    29]])

# L1-L2 Embedders

Με την χρήση αυτώ των embedders βλέπουμε το μοντέλο να χειροτερεύει μιας και πλεόν 
κάνει περισσότερα λάθη όπου προβλέπει την ύπαρξη της ακμής εκεί που δεν υπάρχει
και προβλέπει την απουσία ακμών εκεί που υπάρχουν. 

Δηλαδή βλέπουμε αύξηση στην δευτερεύουσα διαγώνιο του confusion matrix.

Αυτό μπορέι να συμβαίνει μιας και τόσο η l1 όσο και η l2 έχουν την τάση είτε να εκμηδενίζουν είτε να αποδυναμώνουν παραμέτρους σε ένα μοντέλο.

In [39]:
# Weighted-L1 embeddings
from node2vec.edges import WeightedL1Embedder

l1_edges_embs = WeightedL1Embedder(keyed_vectors=n2v_model.wv)

x = [l1_edges_embs[(str(i),str(j))] for i,j in zip(df_expanded2['source'], df_expanded2['target'])]
X_train, X_test, y_train, y_test = train_test_split(x, df_expanded2['link'], random_state = 0)  #uncomment and complete

classifier = RandomForestClassifier(max_depth=5, random_state=0) 
classifier.fit(X_train,y_train)   #uncomment and complete

sklearn.metrics.confusion_matrix(y_test, classifier.predict(X_test))

array([[22913,     5],
       [   44,    13]])

In [40]:
# Weighted-L2 embeddings
from node2vec.edges import WeightedL1Embedder

l1_edges_embs = WeightedL1Embedder(keyed_vectors=n2v_model.wv)

x = [l1_edges_embs[(str(i),str(j))] for i,j in zip(df_expanded2['source'], df_expanded2['target'])]
X_train, X_test, y_train, y_test = train_test_split(x, df_expanded2['link'], random_state = 0)  #uncomment and complete

classifier = RandomForestClassifier(max_depth=5, random_state=0) 
classifier.fit(X_train,y_train)   #uncomment and complete

sklearn.metrics.confusion_matrix(y_test, classifier.predict(X_test))

array([[22913,     5],
       [   44,    13]])